In [ ]:
from transformers import AutoTokenizer, FlaxGPT2LMHeadModel, GPT2Config

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
config = GPT2Config()
# Apparently GPT2 ties the last linear layer to the initial word embeddings
# so the final layer is wte.embedding.T (768, 50257)
# config.tie_word_embeddings = False
hf_model = FlaxGPT2LMHeadModel.from_pretrained("openai-community/gpt2")

inputs = tokenizer("Hello, my dog is cute", return_tensors="np")
outputs = hf_model(**inputs)

# retrieve logts for next token
next_token_logits = outputs.logits[:, -1]

In [102]:
next_token_logits.shape

(1, 50257)

In [1]:
def get_keys(params):
    out = []
    def print_keys_flat(params, prefix=""):
        for p in params:
            if isinstance(params[p], dict):
                print_keys_flat(params[p], f"{prefix}.{p}")
            else:
                out.append((f"{prefix}.{p}", params[p].shape))
    print_keys_flat(params)
    return out
            
def print_keys(params, offset=""):
    for p in params:
        if not isinstance(params[p], dict):
            print(offset+p, params[p].shape)
        else:
            print(offset+p)
            print_keys(params[p], offset + "  ")

In [ ]:
import gpt2 as nn
from importlib import reload
reload(nn)
import jax.numpy as jnp
import jax

config = nn.Config()
model = nn.GPT2(config)
params = model.from_pretrained('gpt2')

# model_blank = nn.GPT2(nn.Config())
# dummy = jnp.ones((1,1), dtype=int)
# params = model.init(key,dummy)

In [ ]:
#inputs = jnp.array(inputs['input_ids'])
import tiktoken

enc = tiktoken.get_encoding('gpt2')
key = jax.random.PRNGKey(42069)
key, gen_key = jax.random.split(key)
start = "Hello, I'm a language model,"
inputs = jnp.array(enc.encode(start)).reshape(1,-1)
#out = model.generate(key, params, inputs, 100)
out = model.generate_batch(gen_key, params, inputs, 2)
jit_gen = jax.jit(lambda params, gen_key, inputs, num_new_tokens: model.generate_batch(gen_key, params, inputs, num_new_tokens), static_argnums=(3,))

In [8]:
print(enc.decode(out[0]))

Hello, I'm a language model, and LazyList is a language model. This is the goal of this book. Let's see 1. Learn ruby p talked about 2. Learn Ruby scripting tutorials 3. Learn Scheme text parsing 4. Create some nice 3D objects 5. Build something from the ground up
Multi-language design
Getting Started
Optimisation of cataphysics and immutability in domain constructs
The chemistry classroom course.
Learning Lab Steps
Learning Lab will lie in the traditional I/O vis
